In [1]:
import warnings
warnings.filterwarnings("ignore")
import cv2
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.models as M
import tensorflow.keras.optimizers as O
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from tensorflow.keras.models import model_from_json
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from sklearn.metrics import precision_score,recall_score,f1_score,confusion_matrix,accuracy_score

In [2]:
''' Class for data loader. It takes the training or evaluation data and creates an object of Sequence class 
from Keras. The class takes care of loading batch of images while training the model and discarding them after use.
This class is used to create objects for model without sift features.
'''
class CNN_Loader_with_sift(Sequence):
    def __init__(self, img_paths, targets, batch_size):
        self.img_paths, self.targets = img_paths, targets
        self.batch_size = batch_size
        #self.all_y = []

    def __len__(self):
        return int(np.ceil(len(self.img_paths) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_imgs = self.img_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.targets[idx * self.batch_size:(idx + 1) * self.batch_size]
        images = []
        s_features = []
        for filename in batch_imgs:
            img = cv2.imread("data/"+filename, cv2.IMREAD_COLOR)
            images.append(np.transpose(image.img_to_array(image.load_img("data/"+filename, target_size=(224, 224))),(1,0,2)))
            blur = cv2.blur(img,(50,50))
            d_temp = np.zeros((25600,))
            blur2 = cv2.GaussianBlur(img,(3,3),0)
            absd=cv2.equalizeHist(cv2.cvtColor(cv2.absdiff(blur2,blur),cv2.COLOR_BGR2GRAY))
            saliency = cv2.saliency.StaticSaliencyFineGrained_create()
            (success, saliencyMap) = saliency.computeSaliency(absd)
            threshMap = cv2.threshold(saliencyMap.astype("uint8"), 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
            ss = saliencyMap.copy()
            ss[:40, :] = 0
            ss[:, :20] = 0
            ss[:, -20:] = 0
            sift1 = cv2.xfeatures2d.SIFT_create(200)
            kp1, desc = sift1.detectAndCompute(ss, None)
            if desc is not None:
                f = desc.flatten()
                d_temp[:min(25600,len(f))] = f[:25600]
            s_features.append(d_temp)
        s_features = np.asarray(s_features)
        return [np.array(images), s_features], np.array(batch_y)

''' Class for data loader. It takes the training or evaluation data and creates an object of Sequence class 
from Keras. The class takes care of loading batch of images while training the model and discarding them after use.
This class is used to create objects for model with sift features.
'''
class CNN_Loader(Sequence):
    def __init__(self, img_paths, targets, batch_size):
        self.img_paths, self.targets = img_paths, targets
        self.batch_size = batch_size
        #self.all_y = []

    def __len__(self):
        return int(np.ceil(len(self.img_paths) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_imgs = self.img_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.targets[idx * self.batch_size:(idx + 1) * self.batch_size]
        images = []
        for filename in batch_imgs:
            images.append(np.transpose(image.img_to_array(image.load_img("data/"+filename, target_size=(224, 224))),(1,0,2)))
        return np.array(images), np.array(batch_y)

In [3]:
def weighted_binary_crossentropy(y_true, y_pred):
    b_ce = K.binary_crossentropy(y_true, y_pred)
    weight_vector = y_true * 0.9 + (1. - y_true) * 0.1
    weighted_b_ce = weight_vector * b_ce
    return K.mean(weighted_b_ce)

In [8]:
''' Method to build a model with sift. Here the input features is passed through 
multi layer dense network upon which global average pooling is done.
The output is finally concatenated with image output from the base model 
and the result is activated by a sigmoid function.
'''
def build_model_with_sift():
    i1 = L.Input((25600))
    x1 = L.Dense(16, activation='relu')(i1)
    x1 = L.Dropout(0.2)(x1)
    x1 = L.Dense(32, activation='relu')(x1)
    x1 = L.Dropout(0.2)(x1)
    x1 = L.Dense(64, activation='relu')(x1)
    x1 = L.Dropout(0.2)(x1)
    base_model = Xception(weights='imagenet', include_top=False)
    x = base_model.output
    x = L.GlobalAveragePooling2D()(x)
    x = L.Concatenate()([x,x1])
    predictions = L.Dense(1, activation='sigmoid')(x)
    model = M.Model(inputs=[base_model.input,i1], outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = True
    adam = O.Adam(learning_rate=0.0001)
    model.compile(optimizer=adam, loss="binary_crossentropy", metrics=['accuracy'])
    return model


''' Method to build a model using pre trained Xception model '''
def build_model():
    base_model = Xception(weights='imagenet', include_top=False)
    x = base_model.output
    x = L.GlobalAveragePooling2D()(x)
    predictions = L.Dense(1, activation='sigmoid')(x)
    model = M.Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = True
    adam = O.Adam(learning_rate=0.0001)
    model.compile(optimizer=adam, loss="binary_crossentropy", metrics=['accuracy'])
    return model

''' Trains the model '''
def training_model(train_data_gen, eval_data_gen, model):
    model.fit_generator(train_data_gen, epochs=3, validation_data=eval_data_gen)
    return model

''' Predicts using the model '''    
def predicting_model(eval_data_gen, model):
    pred=model.predict_generator(eval_data_gen)
    return pred


''' Scores the model predictions '''
def scores(pred, all_y):
    accuracy = accuracy_score(all_y,pred)
    precision = precision_score(all_y, pred)
    recall = recall_score(all_y, pred)
    f1 = f1_score(all_y, pred)
    return {'accuracy':accuracy,'precision':precision,'recall':recall}

In [9]:

''' Splits the data based on given ratio '''
def data_train_test(data, split=0.6):
    train_split = int(split*data.shape[0])
    x_train,y_train = data.iloc[:train_split,0].values, data.iloc[:train_split,1].astype('int').values
    x_test,y_test = data.iloc[train_split:,0].values, data.iloc[train_split:,1].astype('int').values
    return x_train, y_train, x_test, y_test

In [10]:
''' Master function to execute training for a given task with/without sift '''
def execute(task, with_sift=False):
    data = pd.read_csv("./data/"+task+"_training_data.csv",header=None)
    data=data.sample(frac=1)
    x_train, y_train, x_test, y_test = data_train_test(data, 0.5)
    if with_sift:
        model = build_model_with_sift()
        train_data_gen = CNN_Loader_with_sift(x_train, y_train, 32)
        eval_data_gen = CNN_Loader_with_sift(x_test, y_test, 32)
    else:
        model = build_model()
        train_data_gen = CNN_Loader(x_train, y_train, 32)
        eval_data_gen = CNN_Loader(x_test, y_test, 32)
    model.summary()
    model = training_model(train_data_gen, eval_data_gen, model)
#     pred = predicting_model(train_data_gen, model)
    pred1 = predicting_model(eval_data_gen, model)
#     train_scores = scores(np.round(pred), train_data_gen.targets)
    eval_scores = scores(np.round(pred1), eval_data_gen.targets)
#     train_conf_mat = confusion_matrix(np.round(pred),train_data_gen.targets)
    eval_conf_mat = confusion_matrix(np.round(pred1),eval_data_gen.targets)
    
#     print(train_scores, eval_scores)
#     print(train_conf_mat)
    print(eval_conf_mat)
    
    sift = "sift_" if with_sift else ""
    
    model_json = model.to_json()
    with open("Models/CNN_model_xception_"+sift+"full_"+task+".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("./Models/CNN_model_xception_"+sift+"full_"+task+".h5")

In [11]:
execute("jumpshot",with_sift=False)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

Epoch 1/3
257/257 [==============================] - 459s 2s/step - loss: 0.1256 - acc: 0.9510 - val_loss: 0.0359 - val_acc: 0.9898
Epoch 2/3
257/257 [==============================] - 439s 2s/step - loss: 0.0152 - acc: 0.9967 - val_loss: 0.0174 - val_acc: 0.9951
Epoch 3/3
257/257 [==============================] - 439s 2s/step - loss: 0.0064 - acc: 0.9982 - val_loss: 0.0208 - val_acc: 0.9928
[[7433    9]
 [  50  730]]


In [ ]:
del model
K.clear_session()